<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 6.5
## Feature Selection

### Data

**Predict the onset of diabetes based on diagnostic measures.**

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

[Pima Indians Diabetes Database](https://www.kaggle.com/uciml/pima-indians-diabetes-database/download)

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression

#### 1. Load Data

In [2]:
# Read Data
diabetes_csv = '../DATA/diabetes.csv'
diabetes = pd.read_csv(diabetes_csv)

#### 2. Perform EDA

Perform EDA. Check Null Values. Impute if necessary.

In [3]:
diabetes.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


#### 3. Set Target

- Set `Outcome` as target.
- Set Features

In [5]:
X = diabetes.drop(columns=['Outcome']).copy()
y = diabetes['Outcome']

#### 4. Select Feature

The classes in the `sklearn.feature_selection` module can be used for feature selection/dimensionality reduction on sample sets, either to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets.

##### 4.1 Univariate Selection

Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator. 

⭐ Tranform:
- Removes columns for unused features
- Scales remaining data to identical range (typically [0, 1])
- Does some other stuff?

SelectKBest

- SelectKBest removes all but the  highest scoring features
- Use sklearn.feature_selection.chi2 as score function
    > Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification.


More Reads:
[Univariate feature selection](https://scikit-learn.org/stable/modules/feature_selection.html)

- Create an instance of SelectKBest
    - Use sklearn.feature_selection.chi2 as score_func
    - Use k of your choice
- Fit X, y
- Find top 4 features
- Transform features to a DataFrame

In [6]:
# Create an instance of SelectKBest

from sklearn.feature_selection import SelectKBest, chi2

kbest = SelectKBest(k=4, score_func=chi2)

In [7]:
# Fit
kbest_fit = kbest.fit(X, y)
scores = pd.DataFrame(kbest_fit.scores_, index=X.columns, columns=['score'])

scores

,score
Pregnancies,111.519691
Glucose,1411.887041
BloodPressure,17.605373
SkinThickness,53.108040
Insulin,2175.565273
BMI,127.669343
DiabetesPedigreeFunction,5.392682
Age,181.303689


In [8]:
# Print Score
# Find Top 4 Features
top4 = scores.sort_values(by='score', ascending=False).iloc[0:4, :]
top4

,score
Insulin,2175.565273
Glucose,1411.887041
Age,181.303689
BMI,127.669343


In [9]:
# Transform X to Features
transform = kbest_fit.transform(X)

In [10]:
# Transform features to a dataframe
features_df = pd.DataFrame(transform)
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    float64
 1   1       768 non-null    float64
 2   2       768 non-null    float64
 3   3       768 non-null    float64
dtypes: float64(4)
memory usage: 24.1 KB


In [11]:
features_df.head()

,0,1,2,3
0,148.0,0.0,33.6,50.0
1,85.0,0.0,26.6,31.0
2,183.0,0.0,23.3,32.0
3,89.0,94.0,28.1,21.0
4,137.0,168.0,43.1,33.0


##### 4.2 Recursive feature elimination

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

More Reads:
[Recursive feature elimination](https://scikit-learn.org/stable/modules/feature_selection.html)

- Use RFE to extract feature
    - use LogisticRegression as estimator
    - Number of n_features_to_select as of your choice
- Fit X, y to RFE
- Find Selected Features

In [16]:
# ANSWER
model = LogisticRegression(max_iter=1000, solver = 'lbfgs')
rfe = RFE(estimator=model, n_features_to_select=4)
fit = rfe.fit(X,y)
rfe.ranking_


array([1, 1, 3, 5, 4, 1, 1, 2])

👉🏼 `.support_` is a boolean array that indicates whether a feature was selected or not.  It is one of the most important.

In [26]:
# Print Score

scores = pd.DataFrame(rfe.support_, index=X.columns, columns=['score'])

top4 = scores.sort_values(by='score', ascending=True).iloc[0:4, :]

scores[scores.score]
# Find Features

,score
Pregnancies,True
Glucose,True
BMI,True
DiabetesPedigreeFunction,True




---



---



> > > > > > > > > © 2024 Institute of Data


---



---



